DIVERSE (Diverse Verifier on Reasoning Step)

Paper : https://arxiv.org/pdf/2206.02336

github :  https://github.com/microsoft/CodeT/tree/main/DIVERSE

Refrences : https://github.com/marcellusruben/medium-resources/tree/main

Các bước thực hiện :

**Bộ xác minh đa dạng trên từng bước suy luận (DIVERSE)**  

Hình 1 mô tả tổng quan về DIVERSE. Các ý tưởng chính bao gồm ba điểm sau:  
1. **Tận dụng các prompt đa dạng** để tạo ra nhiều lộ trình suy luận khác nhau từ mô hình ngôn ngữ (Mục 2.1).  
2. **Huấn luyện các bộ xác minh** để tổng hợp câu trả lời từ nhiều lộ trình suy luận (Mục 2.2).  
3. **Sử dụng độ chính xác từng bước suy luận** để cải thiện hiệu quả của bộ xác minh bằng cách bỏ phiếu (Mục 2.3).  

**2.1 Prompt đa dạng**  
Để suy luận hiệu quả, việc khám phá các lộ trình suy luận đa dạng là rất hữu ích, dựa trên ý tưởng rằng "Mọi con đường đều dẫn tới thành Rome". Wang et al. (2022c) đã đề xuất tạo ra nhiều lộ trình suy luận từ các mô hình ngôn ngữ bằng cách sử dụng phương pháp giải mã có tính ngẫu nhiên (sampling decoding). Tuy nhiên, phương pháp của họ dựa vào một tập hợp cố định các ví dụ minh họa (exemplars) cho tất cả các prompt, điều này có thể gây ra thiên lệch và hạn chế tính đa dạng của các lộ trình suy luận được tạo ra.  

Để giải quyết vấn đề này, chúng tôi chọn ngẫu nhiên **M1** prompt khác nhau cho mỗi câu hỏi và sau đó sinh ra **M2** lộ trình suy luận cho mỗi prompt bằng cách sử dụng giải mã ngẫu nhiên. Bằng cách này, chúng tôi thu được tổng cộng **M = M1 × M2** lộ trình suy luận đa dạng cho mỗi câu hỏi.  

 **2.2 Bộ xác minh bằng cách bỏ phiếu**  

**Xác minh (Verifier):**  
Bộ xác minh nhận đầu vào là một câu hỏi và một lộ trình suy luận ứng viên, sau đó đầu ra là xác suất mà lộ trình suy luận đó dẫn đến câu trả lời đúng. Chúng tôi sử dụng mô hình **deberta-v3-large** (He et al., 2021) làm mô hình nền tảng, với một đầu ra vô hướng nhỏ để đưa ra dự đoán dựa trên token [CLS].  

**Huấn luyện bộ xác minh:**  
Đối với mỗi câu hỏi trong tập huấn luyện, chúng tôi tạo ra nhiều lộ trình suy luận ứng viên bằng phương pháp suy luận dạng chuỗi tư duy (chain-of-thought reasoning). Các lộ trình suy luận dẫn đến câu trả lời đúng sẽ được coi là **tích cực**, trong khi các lộ trình khác sẽ được coi là **tiêu cực**.  

**Bộ xác minh bằng cách bỏ phiếu (Voting Verifier):**  
Wang et al. (2022c) đã sử dụng phương pháp bỏ phiếu đa số để tổng hợp các dự đoán từ các lộ trình suy luận khác nhau. Tuy nhiên, phương pháp này có thể thất bại khi phần lớn các lộ trình suy luận bị sai lệch, trong khi một số ít lộ trình suy luận lại hợp lý.  

Chúng tôi đề xuất một phương pháp **bỏ phiếu kết hợp bộ xác minh**, sử dụng cả việc bỏ phiếu và xác suất từ bộ xác minh. Công thức tính như sau:  

$$
\hat{y} = \arg\max_y \sum_{i=1}^M \mathbb{1}_{y_i=y} \cdot f(x_i, z_i, y_i)
$$  

Trong đó:  
- $ \mathbb{1}_{y_i=y} $ là một hàm chỉ báo, trả về 1 nếu $ y_i = y $, ngược lại trả về 0.  
- $ f(\cdot) $ là xác suất do bộ xác minh dự đoán.  

Phương pháp này giúp tận dụng thông tin từ cả số lượng lộ trình suy luận (bỏ phiếu) và chất lượng của từng lộ trình (do bộ xác minh đánh giá).  

**2.3 Bộ xác minh theo bước (Step-aware Voting Verifier)**  

Mỗi lộ trình suy luận bao gồm nhiều bước. Chúng tôi giả định rằng không phải tất cả các bước trong một lộ trình suy luận sai đều sai ở mức độ như nhau, và một số bước vẫn có thể hữu ích cho quá trình suy luận. Để khai thác điều này, chúng tôi mở rộng bộ xác minh bằng cách bỏ phiếu thành **bộ xác minh theo bước (step-aware voting verifier)**, với việc giới thiệu một hàm mất mát mở rộng:  

$$
L = L_0 + \alpha \cdot L_1
$$

Trong đó:  
- $ L_0 $: Mất mát ban đầu từ bộ xác minh.  
- $ L_1 $: Mất mát phụ trợ ở mức từng bước, được tính bằng:  
  $$
  L_1 = \sum_{i=1}^{|\hat{D}|} \sum_{j=1}^{|S_i|} \text{BCE}(\text{label}_{i,j}, f'(input_{i,j}))
  $$  
  - $ \alpha $: Siêu tham số để cân bằng giữa mất mát ban đầu $ L_0 $ và mất mát phụ trợ theo bước $ L_1 $.  
  - $ S_{i,1}, S_{i,2}, ..., S_{i,|S_i|} $: Các bước trong lộ trình suy luận $ z_i $.  
  - $ \text{label}_{i,j} $: Nhãn cho biết bước $ S_{i,j} $ là đúng hay sai.  
  - $ f'(input_{i,j}) $: Xác suất nhãn tích cực cho bước $ S_{i,j} $, được dự đoán bởi mô hình.  

**Tạo nhãn ở mức bước (step-level labels):**  
Để tạo nhãn ở mức bước (tức là $ \text{label}_{i,j} $) cho các dữ liệu huấn luyện tiêu cực có câu trả lời sai, chúng tôi thiết kế một thuật toán so sánh các kết quả trung gian giữa các bước trong lộ trình suy luận tích cực và tiêu cực.  

Hình 3 minh họa thuật toán này, với khả năng áp dụng không chỉ cho các bài toán toán học mà còn mở rộng sang các nhiệm vụ suy luận khác. Chúng tôi sử dụng một mô hình **suy luận ngôn ngữ tự nhiên (natural language inference)** có sẵn, **roberta-large-mnli** (Liu et al., 2019), để kiểm tra xem hai bước suy luận có tương đương về mặt ngữ nghĩa hay không.  

Đối với một bước suy luận nhất định, nếu không thể tìm thấy bất kỳ bước nào tương đương về mặt ngữ nghĩa trong các lộ trình suy luận tích cực, chúng tôi sẽ gán nhãn bước đó và tất cả các bước sau nó là **các bước tiêu cực**.  

In [ ]:
!pip install instructor datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.7.1
    Uninstalling jiter-0.7.1:
      Successfully uninstalled jiter-0.7.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [ ]:
import json
import random
import instructor
import re
from pathlib import Path
from typing import List, TextIO

import openai
from datasets import load_dataset
# from fire import Fire
from pydantic import BaseModel
from tqdm import tqdm


def remove_substrings_with_double_angle_brackets(input_string):
    # Define the pattern to match substrings within double angled brackets
    pattern = r"<<[^>]+>>"
    # Use the sub() function from the re module to replace matching substrings with an empty string
    result = re.sub(pattern, "", input_string)
    return result


class ReasonSample(BaseModel):
    question: str
    explanation: str = ""
    answer: str = ""
    wrong_explanation: str = ""
    wrong_answer: str = ""
    pred: str = ""


class ReasonData(BaseModel):
    samples: List[ReasonSample]

    @classmethod
    def load(cls, path: str):
        samples = []
        with open(path) as f:
            for line in f:
                raw = json.loads(line)
                samples.append(ReasonSample(**raw))

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_test(cls, path: str = "gsm8k", subset: str = "main", split="test"):
        samples = []
        for raw in load_dataset(path, subset, split=split):
            explanation, answer = raw["answer"].split("####")
            explanation = remove_substrings_with_double_angle_brackets(explanation)
            samples.append(
                ReasonSample(
                    question=raw["question"].strip(),
                    explanation=explanation.strip(),
                    answer=answer.strip(),
                )
            )

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_incoherent_objects(cls, split: str = "test", sample: bool = False):
        if split == "train" and sample:
            samples = [
                ReasonSample(
                    question="There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
                    explanation="There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.",
                    answer="6",
                    wrong_explanation="There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.",
                    wrong_answer="21",
                ),
                ReasonSample(
                    question="If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??",
                    explanation="There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.",
                    answer="5",
                    wrong_explanation="There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot",
                    wrong_answer="2",
                ),
                ReasonSample(
                    question="Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
                    explanation="Originally, Leah had 32 chocolates and her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39 pieces left in total.",
                    answer="39",
                    wrong_explanation="Originally, Leah had 32 + 42 = 74 chocolates and her sister had 32. So in total they had 74 - 35 = 39. After eating 35, they had 42 pieces left in total",
                    wrong_answer="42",
                ),
                ReasonSample(
                    question="Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
                    explanation="Jason had 20 lollipops originally. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8 lollipops.",
                    answer="8",
                    wrong_explanation="Jason had 20 - 12 = 8 lollipops originally. Then he had 20 after giving some to Denny. So he gave Denny 12 lollipops",
                    wrong_answer="12",
                ),
            ]
            return cls(samples=samples)
        else:
            return cls.load_gsm8k_test()

    @classmethod
    def load_from_name(cls, name: str, **kwargs):
        if name == "gsm8k":
            return cls.load_gsm8k_incoherent_objects(**kwargs)
        else:
            raise KeyError(name)


In [ ]:
data_name = 'gsm8k'
data_test = ReasonData.load_from_name(data_name, split="test", sample=True)
data_test.samples = random.sample(data_test.samples, k=5)

examples_data = [f"Q: {data.question} \n A: {data.explanation}" for data in data_test.samples]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
examples_data

['Q: Tomorrow, 42 adults and 15 babies will be attending a function at Mia’s restaurant. The restaurant has 5 times as many regular chairs as high chairs. If there are 8 high chairs, how many more chairs does she have to get? \n A: Since there are 8 high chairs, the number of regular chairs is 5 * 8 = 40 regular chairs\nThe total number of both types of chairs is therefore 8 + 40 = 48 chairs\nHowever, the total number of attendants will be 42 + 15 = 57 people\nMia will therefore need 57-48 = 9 more chairs',
 "Q: A shoe store was having a weekend sale on a brand of popular tennis shoes. On Friday the store sold 14 pairs of tennis shoes. The next day they sold double that number of shoes. On the last day of the sale they sold one-half the amount that they did the day before, but six people returned their pairs because they didn't fit. How many tennis shoes were sold by the end of the sale? \n A: On Saturday 14*2=28 shoes were sold\nOn Sunday 28/2=14 shoes were sold\nBefore the returns we

In [ ]:
len(examples_data)

5

In [ ]:
import instructor
from openai import AsyncOpenAI
from pydantic import BaseModel
from typing import Literal
from textwrap import dedent
import asyncio
from collections import defaultdict

client = instructor.from_openai(AsyncOpenAI(api_key='sk-RYiDonFCU85Bfr242X2POvCBhlhfHlz8knZQoVjWVJw180s6',
                                              base_url="https://api.chatanywhere.tech/v1"))


class Response(BaseModel):
    chain_of_thought: str
    answer: int

async def generate_response(query: str, examples: list[str]):
    formatted_examples = "\n".join(examples)
    return await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": dedent(
                    f"""
                You are a world class AI that excels at answering
                user queries in a succint and accurate manner.

                <query>
                {query}
                </query>

                <examples>
                {formatted_examples}
                </examples>
                """
                ),
            }
        ],
        response_model=Response,
    )

async def generate_response_batch(
    query: str, examples: list[str], n_examples_per_batch: int
):
    batches: list[list[str]] = []
    for i in range(0, len(examples), n_examples_per_batch):
        batches.append(examples[i : i + n_examples_per_batch])

    coros = [generate_response(query, example_batch) for example_batch in batches]
    return await asyncio.gather(*coros)

In [ ]:
import nest_asyncio
query = """Betty is saving money for a new wallet which
costs $100. Betty has only half of the money she needs.
Her parents decided to give her $15 for that purpose,
and her grandparents twice as much as her parents. How
much more money does Betty need to buy the wallet?"""

loop = asyncio.get_event_loop()

# Apply nest_asyncio to the loop
nest_asyncio.apply(loop)

# Run the coroutine using the loop's run_until_complete method
generated_responses = loop.run_until_complete(generate_response_batch(query, examples_data, 1))

In [ ]:
generated_responses

[Response(chain_of_thought='Betty needs $100 for a new wallet and currently has half of that amount, which is $50. Her parents give her $15, and her grandparents give her twice that amount, which is $30. So, the total amount Betty receives from her parents and grandparents is $15 + $30 = $45. Now we need to calculate how much money Betty will have after receiving this extra amount: $50 (current savings) + $45 (received) = $95. She still needs $100 - $95 = $5 more to buy the wallet.', answer=5),
 Response(chain_of_thought='Betty needs $100 for the wallet and she currently has half of that, which is $100 / 2 = $50. Her parents gave her $15, and her grandparents gave her $15 * 2 = $30. Therefore, she has $50 + $15 + $30 = $95 in total. To find out how much more she needs, we subtract what she has from the total cost of the wallet: $100 - $95 = $5. Thus, Betty needs $5 more to buy the wallet.', answer=5),
 Response(chain_of_thought='Betty needs a total of $100 for the wallet. Currently, sh

In [ ]:
class Grading(BaseModel):
    grade: Literal["Poor", "Average", "Good", "Excellent"]

    def get_score(self):
        mapping = {
            "Poor": 0.25,
            "Average": 0.5,
            "Good": 0.75,
            "Excellent": 1,
        }
        return mapping[self.grade]

async def score_response(query: str, response: Response) -> tuple[Response, Grading]:
    return (
        response,
        await client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": dedent(
                        f"""
                You are a world class AI that excels at grading
                responses to a user query in a succint and clear
                manner based on chain of thought.

                <query>
                {query}
                </query>

                <response>
                {response}
                </response>
                """
                    ),
                }
            ],
            response_model=Grading,
        ),
    )

async def score_responses(
    query: str, responses: list[Response]
) -> list[tuple[Response, Grading]]:
    coros = [score_response(query, response) for response in responses]
    return await asyncio.gather(*coros)

In [ ]:
scored_responses = asyncio.run(score_responses(query, generated_responses))

In [ ]:
scored_responses

[(Response(chain_of_thought='Betty needs $100 for a new wallet and currently has half of that amount, which is $50. Her parents give her $15, and her grandparents give her twice that amount, which is $30. So, the total amount Betty receives from her parents and grandparents is $15 + $30 = $45. Now we need to calculate how much money Betty will have after receiving this extra amount: $50 (current savings) + $45 (received) = $95. She still needs $100 - $95 = $5 more to buy the wallet.', answer=5),
  Grading(grade='Excellent')),
 (Response(chain_of_thought='Betty needs $100 for the wallet and she currently has half of that, which is $100 / 2 = $50. Her parents gave her $15, and her grandparents gave her $15 * 2 = $30. Therefore, she has $50 + $15 + $30 = $95 in total. To find out how much more she needs, we subtract what she has from the total cost of the wallet: $100 - $95 = $5. Thus, Betty needs $5 more to buy the wallet.', answer=5),
  Grading(grade='Excellent')),
 (Response(chain_of_t

In [ ]:
scores: dict[int, float] = defaultdict(int)

for response, grade in scored_responses:
    scores[response.answer] += grade.get_score()

print(scores)
#> defaultdict(<class 'int'>, {5: 3.5})

answer = max(scores, key=scores.get)
print(answer)

defaultdict(<class 'int'>, {5: 4.5})
5


In [ ]:
from openai import AsyncOpenAI
import instructor
from pydantic import BaseModel
from typing import Callable, Literal
from textwrap import dedent
import math
import nest_asyncio
import asyncio


class Response(BaseModel):
    chain_of_thought: str
    response: Literal["A", "B"]
    confidence: Literal[
        "Very High Confidence",
        "High Confidence",
        "Moderate Confidence",
        "Low Confidence",
        "Very Low Confidence",
    ]

    def generate_score(self) -> float:
        confidence_scores = {
            "Very High Confidence": 1,
            "High Confidence": 0.8,
            "Moderate Confidence": 0.6,
            "Low Confidence": 0.4,
            "Very Low Confidence": 0.2,
        }
        return confidence_scores[self.confidence]


client = AsyncOpenAI(api_key='sk-RYiDonFCU85Bfr242X2POvCBhlhfHlz8knZQoVjWVJw180s6',
                                              base_url="https://api.chatanywhere.tech/v1")
client = instructor.from_openai(client)

def prompt_template_1(question: str, options: list[str]):
    assert len(options) == 2
    a, b = options

    return dedent(
        f"""
    You are a world class AI System which excels at understanding complex user stories and generating responses. Output your prediction and also quantify your confidence in your prediction with the following scale.

    - Very High Confidence: The model is highly confident in its prediction, displaying deep understanding, flawless execution, and no noticeable errors.
    - High Confidence: The model is confident in its prediction, with strong relevance and minor errors that do not detract from overall quality.
    - Moderate Confidence: The model has moderate confidence in its prediction, which is generally relevant with some inaccuracies, and meets minimum requirements.
    - Low Confidence: The model has low confidence in its prediction, with limited relevance and several inaccuracies.
    - Very Low Confidence: The model has very low confidence in its prediction, which is largely irrelevant, inaccurate, or incomplete, needing significant improvement


    Context
    {question}

    Options
    A. {a}
    B. {b}
    """
    )


def prompt_template_2(question: str, options: list[str]):
    assert len(options) == 2
    a, b = options

    return dedent(
        f"""
    <prompt>
        <Task>
        You are about to be passed a story. You are to select the correct response from the options provided.

         <confidence-levels>
             <level>
                 <name>Very High Confidence</name>
                 <description>The model is highly confident in its prediction, displaying deep understanding, flawless execution, and no noticeable errors.</description>
             </level>
             <level>
                 <name>High Confidence</name>
                 <description>The model is confident in its prediction, with strong relevance and minor errors that do not detract from overall quality.</description>
             </level>
             <level>
                 <name>Moderate Confidence</name>
                 <description>The model has moderate confidence in its prediction, which is generally relevant with some inaccuracies, and meets minimum requirements.</description>
             </level>
             <level>
                 <name>Low Confidence</name>
                 <description>The model has low confidence in its prediction, with limited relevance and several inaccuracies.</description>
             </level>
             <level>
                 <name>Very Low Confidence</name>
                 <description>The model has very low confidence in its prediction, which is largely irrelevant, inaccurate, or incomplete, needing significant improvement</description>
             </level>
         </confidence-levels>
        </Task>

        <Question>
        {question}
        </Question>

        <Options>
        <option>A: {a}</option>
        <option>B: {b}</option>
        </Options>
    </prompt>
    """
    )


async def generate_response(
    question: str, options: list[str], prompt_template: Callable[[str, list[str]], str]
):
    return await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt_template(question, options),
            }
        ],
        response_model=Response,
    )


async def generate_responses(
    questions: list[str], prompt_template: Callable[[str, list[str]], str]
):
    return await asyncio.gather(
        *[
            generate_response(
                question=question["question"],
                options=question["options"],
                prompt_template=prompt_template,
            )
            for question in questions
        ]
    )


def calculate_entropy(probs: list[float]) -> float:
    return sum([p * math.log(p) if p != 0 else 0 for p in probs])


def calculate_mutual_information(predictions: list[Response]) -> float:
    probs = [
        [prediction.generate_score(), 1 - prediction.generate_score()]
        for prediction in predictions
    ]

    avg_probs = [0, 0]

    for p1, p2 in probs:
        avg_probs[0] += p1
        avg_probs[1] += p2

    h_marginal = calculate_entropy([i / len(probs) for i in avg_probs])
    h_conditional = sum([calculate_entropy(prob) for prob in probs]) / len(probs)

    return h_marginal - h_conditional


if __name__ == "__main__":
    queries = [
        {
            "question": "Karen was assigned a roommate her first year of college. Her roommate asked her to go to a nearby city for a concert. Karen agreed happily. The show was absolutely exhilarating.",
            "options": [
                "Karen became good friends with her roommate.",
                "Karen hated her roommate.",
            ],
        },
        {
            "question": "Jim got his first credit card in college. He didn’t have a job so he bought everything on his card. After he graduated he amounted a $10,000 debt. Jim realized that he was foolish to spend so much money.    ",
            "options": [
                "Jim decided to devise a plan for repayment.",
                "Jim decided to open another credit card.",
            ],
        },
        {
            "question": "Gina misplaced her phone at her grandparents. It wasn’t anywhere in the living room. She realized she was in the car before. She grabbed her dad’s keys and ran outside.",
            "options": [
                "She found her phone in the car.",
                "She didn’t want her phone anymore.",
            ],
        },
    ]

    best_mi_score = float("-inf")
    best_template = None
    loop = asyncio.get_event_loop()

    # Apply nest_asyncio to the loop
    nest_asyncio.apply(loop)

    # Run the coroutine using the loop's run_until_complete method

    for prompt_template in [prompt_template_1, prompt_template_2]:
        responses = loop.run_until_complete(generate_responses(queries, prompt_template))
        mi_score = calculate_mutual_information(responses)
        print(f"{prompt_template.__name__}: {mi_score}")
        #> prompt_template_1: -0.0781292189485728
        #> prompt_template_2: -0.05907285153542691
        if mi_score > best_mi_score:
            best_mi_score = mi_score
            best_template = prompt_template.__name__

    print(best_template, best_mi_score)
    #> prompt_template_2 -0.05907285153542691

In [ ]:
probs = [
    [prediction.generate_score(), 1 - prediction.generate_score()]
    for prediction in responses
]

In [ ]:
avg_probs = [0, 0]

for p1, p2 in probs:
    avg_probs[0] += p1
    avg_probs[1] += p2
